In [1]:
import math
import numpy as np
import os
import pandas as pd
import pickle
from PIL import Image
from sklearn.metrics import confusion_matrix, matthews_corrcoef
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer, GPT2Model, pipeline, AutoModelForCausalLM, \
AutoTokenizer, BitsAndBytesConfig, LlamaTokenizer, LlamaModel, AutoModelForTextEncoding
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from transformers import RobertaModel, RobertaTokenizer
import random
seed_value = 42
torch.manual_seed(seed_value)
np.random.seed(seed_value)

random.seed(seed_value)


In [2]:
tokenizer = AutoTokenizer.from_pretrained("/data/kaushik3/models--pszemraj--flan-ul2-text-encoder/snapshots/1ffcec2cc23aa34ea6edb96f86c12f179ee9e87f")
model = AutoModelForTextEncoding.from_pretrained("/data/kaushik3/models--pszemraj--flan-ul2-text-encoder/snapshots/1ffcec2cc23aa34ea6edb96f86c12f179ee9e87f")
    

from datasets import load_dataset
dataset = load_dataset("yaful/DeepfakeTextDetect")

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/home/csgrad/sunilruf/miniconda3/envs/bio3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
human_name = 'squad_human'
human_train = [data for data in dataset['train'] if data['src'] == human_name]
human_test = [data for data in dataset['test'] if data['src'] == human_name]
human_val = [data for data in dataset['validation'] if data['src'] == human_name]

llm_train = [data for data in dataset['train'] if 'flan' in data['src'] and 'squad' in data['src']]
llm_test = [data for data in dataset['test'] if 'flan' in data['src'] and 'squad' in data['src']]
llm_val = [data for data in dataset['validation'] if 'flan' in data['src'] and 'squad' in data['src']]

import random

min_train_length = min(len(human_train), len(llm_train))
min_test_length = min(len(human_test), len(llm_test))
min_val_length = min(len(human_val), len(llm_val))

squad_human_train = random.sample(human_train, min_train_length)
squad_human_test = random.sample(human_test, min_test_length)
squad_human_val = random.sample(human_val, min_val_length)

squad_gpt_train = random.sample(llm_train, min_train_length)
squad_gpt_test = random.sample(llm_train, min_test_length)
squad_gpt_val = random.sample(llm_val, min_val_length)

print(len(human_train), len(llm_train))
print(len(squad_human_train), len(squad_gpt_train))
print(len(squad_human_test), len(squad_gpt_test))
print(len(squad_human_val), len(squad_gpt_val))

15820 3885
3885 3885
486 486
486 486


In [4]:
import pandas as pd

Human = pd.DataFrame()
squad_human = squad_human_train + squad_human_val + squad_human_test
Human['text'] = [data['text'] for data in squad_human]

AI = pd.DataFrame()
squad_gpt = squad_gpt_train + squad_gpt_val + squad_gpt_test
AI['text'] = [data['text'] for data in squad_gpt]

In [6]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to('cuda')
tokenizer.add_special_tokens({'pad_token': '[PAD]'}) 
def generate_bert_embeddings(text):
    input_ids = tokenizer.encode(text, return_tensors='pt',padding=True, truncation=True, max_length=512).to(device) 
    #inputs = {name: tensor.to('cuda') for name, tensor in inputs.items()}  # Move inputs to GPU
    with torch.no_grad():
        outputs = model(input_ids)
        #embeddings = torch.mean(outputs.last_hidden_state, dim=1)  # Mean pooling of token embeddings
        #last_hidden_states = outputs.last_hidden_state
        embeddings = outputs.last_hidden_state.mean(dim = 1).squeeze().to('cpu')
    return embeddings.numpy()
import numpy as np

AI['embeddings'] = AI['text'].apply(generate_bert_embeddings)
Human['embeddings'] = Human['text'].apply(generate_bert_embeddings)

In [8]:
AI['embeddings'][0].shape

(4096,)

In [2]:
Human_embeddings = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Flan/human_embeddings.npy', allow_pickle=True)
AI_embeddings = np.load('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Flan/opt_embeddings.npy', allow_pickle=True)
Human_embeddings = Human_embeddings[:len(AI_embeddings)]
reshaped_gpt = []
for i in range(len(AI_embeddings)):
    reshaped = AI_embeddings[i].reshape(1,64,64)
    reshaped_gpt.append(reshaped)
    
reshaped_human = []
for i in range(len(Human_embeddings)):
    reshaped = Human_embeddings[i].reshape(1,64,64)
    reshaped_human.append(reshaped)
    
import torch

reshaped_gpt_tensor = torch.tensor(reshaped_gpt)
reshaped_human_tensor = torch.tensor(reshaped_human)

import torch

# Create tensors with zeros and ones
zeros_tensor = torch.zeros(len(reshaped_gpt))
ones_tensor = torch.ones(len(reshaped_human))

# Concatenate the tensors along the first dimension
labels = torch.cat((zeros_tensor, ones_tensor), dim=0)

# Display the result tensor
print(labels)
embeds = torch.cat((reshaped_gpt_tensor, reshaped_human_tensor), dim=0)

print(len(reshaped_gpt_tensor), len(reshaped_human_tensor), len(embeds), len(labels))

tensor([0., 0., 0.,  ..., 1., 1., 1.])
8098 4857 12955 12955


/tmp/ipykernel_407285/2818229257.py:16: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/torch/csrc/utils/tensor_new.cpp:245.)
  reshaped_gpt_tensor = torch.tensor(reshaped_gpt)


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

test_size = 0.2
dataset = TensorDataset(embeds, labels)

# Split dataset into train and test sets
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)

# Define batch size
batch_size = 64

# Create DataLoader for training set
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
import torch

import torch.nn as nn
import torch.optim as optim

class GPT_CNN2D(nn.Module):
    def __init__(self, embedding_model):
        super(GPT_CNN2D, self).__init__()
        self.em_model = embedding_model
        if embedding_model == "gpt":
            self.conv1 = nn.Conv2d(1, 64, kernel_size=5, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()
            self.fc1 = nn.Linear(1056, 128)
            self.fc2 = nn.Linear(128, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()
        elif embedding_model == "mistral":
            self.conv1 = nn.Conv2d(1, 128, kernel_size=5, padding=1, stride = 2)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.conv2 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(kernel_size=2)
            self.flatten = nn.Flatten()            
            self.fc1 = nn.Linear(3136, 512)
            self.fc2 = nn.Linear(512, 2)
            self.relu = nn.ReLU()
            # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        if self.em_model == "gpt":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            x = self.relu(self.fc1(x))
            x = self.fc2(x)
            # x = self.sigmoid(self.fc2(x))
            return x
        elif self.em_model == "mistral":
            x = self.conv1(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.conv2(x)
            x = self.relu(x)
            x = self.pool(x)
            x = self.flatten(x)
            x = self.relu(self.fc1(x))
            x = self.fc2(x)
            # x = self.sigmoid(self.fc2(x))
            return x


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPT_CNN2D(embedding_model = "mistral").to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
max_val_accuracy = 0
# Step 5: Training Loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).long()
        
        optimizer.zero_grad()
        #print(inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Validation
    model.eval()
    correct = 0
    total = 0
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device).long()
            outputs = model(inputs)
            predicted = torch.argmax(outputs, 1)
            TP += ((predicted == 1) & (labels == 1)).sum().item()
            FP += ((predicted == 1) & (labels == 0)).sum().item()
            TN += ((predicted == 0) & (labels == 0)).sum().item()
            FN += ((predicted == 0) & (labels == 1)).sum().item()
            total += labels.size(0)
            #print(labels, predicted)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    if max_val_accuracy<val_accuracy:
        max_TP = TP
        max_FP = FP
        max_TN = TN
        max_FN = FN
    max_val_accuracy = max(max_val_accuracy, val_accuracy)
    
    
    print(f'Validation Accuracy: {val_accuracy:.4f}')
    print(f'TP: {TP}, FP: {FP}, TN: {TN}, FN: {FN}')


Epoch [1/100], Loss: 0.5457
Validation Accuracy: 0.8086
TP: 843, FP: 358, TN: 1252, FN: 138
Epoch [2/100], Loss: 0.3505
Validation Accuracy: 0.8823
TP: 888, FP: 212, TN: 1398, FN: 93
Epoch [3/100], Loss: 0.1973
Validation Accuracy: 0.9201
TP: 955, FP: 181, TN: 1429, FN: 26
Epoch [4/100], Loss: 0.1234
Validation Accuracy: 0.9506
TP: 940, FP: 87, TN: 1523, FN: 41
Epoch [5/100], Loss: 0.0941
Validation Accuracy: 0.9533
TP: 896, FP: 36, TN: 1574, FN: 85
Epoch [6/100], Loss: 0.0780
Validation Accuracy: 0.9467
TP: 968, FP: 125, TN: 1485, FN: 13
Epoch [7/100], Loss: 0.0582
Validation Accuracy: 0.9637
TP: 932, FP: 45, TN: 1565, FN: 49
Epoch [8/100], Loss: 0.0505
Validation Accuracy: 0.9657
TP: 928, FP: 36, TN: 1574, FN: 53
Epoch [9/100], Loss: 0.0395
Validation Accuracy: 0.9691
TP: 928, FP: 27, TN: 1583, FN: 53
Epoch [10/100], Loss: 0.0322
Validation Accuracy: 0.9695
TP: 936, FP: 34, TN: 1576, FN: 45
Epoch [11/100], Loss: 0.0256
Validation Accuracy: 0.9711
TP: 931, FP: 25, TN: 1585, FN: 50
Epo

In [6]:
print(f"Maximum Validation Accuracy:  {max_val_accuracy:.3f}")
print("TP: ", max_TP)
print("FP: ", max_FP)
print("TN: ", max_TN)
print("FN: ", max_FN)

import math

MCC = (max_TP * max_TN - max_FP * max_FN) / math.sqrt((max_TP + max_FP) * (max_TP + max_FN) * (max_TN + max_FP) * (max_TN + max_FN))

print(f'Matthews Correlation Coefficient (MCC): {MCC:.3f}')

TPR = max_TP / (max_TP + max_FN)
FPR = max_FP / (max_FP + max_TN)

print(f'True Positive Rate (TPR): {TPR:.3f}')
print(f'False Positive Rate (FPR): {FPR:.3f}')


Maximum Validation Accuracy:  0.975
TP:  949
FP:  32
TN:  1578
FN:  32
Matthews Correlation Coefficient (MCC): 0.948
True Positive Rate (TPR): 0.967
False Positive Rate (FPR): 0.020


In [15]:
AI_embeddings = AI['embeddings']

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Flan/flan_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, AI_embeddings)

In [16]:
Human_embeddings = Human['embeddings']

with open('/home/csgrad/sunilruf/detect_llm/sunil_code/LLM/code.old/Deepfake_dataset/Squad/Flan/human_embeddings.npy', 'wb') as f:
    #for i in range(len(gpt['embeddings'])):
    np.save(f, Human_embeddings)